In [ ]:
# PARTE 1. PROCESO DE ETL CON BASE DE DATOS

### Nombre de la asignatura: Mineria de datos

### Carrera: Big Data

### Integrantes: Paula Gomez, Victor Cabrera y Maicol Lojano

### Fecha: 26-4-2024

### Objetivos de la Práctica:

En la siguiente práctica se realizará el proceso de Extracción, Transformación y Carga de Datos.

En primera instancia se debe de crear la estructura de la base de datos para ello se adjunta el script de la base de datos. En mysql cuyo nombre es: Dump20240109.ssql

In [2]:
# Si no tenemos instalado la librería mysql.connector se debe de ejecutar la siguiente línea de comandos.

!pip install mysql.connector

Defaulting to user installation because normal site-packages is not writeable
  Using cached mysql-connector-2.2.9.tar.gz (11.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] No se pudo completar la operación porque el archivo contiene un virus o software potencialmente no deseado while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] No se pudo completar la operación porque el archivo contiene un virus o software potencialmente no deseado



In [3]:
pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
#1. IMPORTAMOS LIBRERIAS NECESARIAS

import mysql.connector
import pandas as pd


In [23]:
# CARGAMOS LA DATA

ruta_archivo_excel="Datos_Taxis.xls"
df=pd.read_excel(ruta_archivo_excel)
df.head()

,Placa,Clase,Marca,Año de fabricación del taxi,Categoría,Tipo taxi
0,AP-000024,Taxi Aeropuerto,Toyota,1990,Automovil,Sedan
1,AP-000033,Taxi Aeropuerto,Nissan,1990,Automovil,Sedan
2,AP-000035,Taxi Aeropuerto,Nissan,1990,Automovil,Sedan
3,AP-000049,Taxi Aeropuerto,Volvo,1990,Automovil,Sedan
4,AP-000074,Taxi Aeropuerto,Nissan,1990,Automovil,Sedan


In [26]:
# CONEXION A LA BASE DE DATOS

# METODO PARA CONECTAR A LA BASE DE DATOS

def establecer_conexion(host, user, password, database):
  try:
      conexion = mysql.connector.connect(
          host = host,
          user = user,
          password = password,
          database = database)
      return conexion
  except mysql.connector.Error as err:
        print(f"Error:{ err}")
        return None

In [27]:
conexion = establecer_conexion("localhost", "root", "", "taxis")
#Comprobación de la conexión
conexion.is_connected()

True

In [9]:
# ELIMINAR LOS DATOS DE LA BASE DE DATOS

def eliminar_datos_base(conexion):
    try:
        # Desactivar las restricciones de clave externa para permitir la eliminación
        #with conexion.cursor() as cursor:
            #cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
        cursor=conexion.cursor()
        # Eliminar datos de las tablas
        delete_queries = [
            'DELETE FROM taxi;',
            'DELETE FROM clase;',
            'DELETE FROM marca;',
            'DELETE FROM categoria;',
            'DELETE FROM tipo_taxi;',
            'DELETE FROM taxi_stg;'
        ]

        for query in delete_queries:
            cursor.execute(query)

        # Reiniciar los contadores de autoincremento
        reset_auto_increment_queries = [
            'ALTER TABLE clase AUTO_INCREMENT = 1;',
            'ALTER TABLE marca AUTO_INCREMENT = 1;',
            'ALTER TABLE marca AUTO_INCREMENT = 1;',
            'ALTER TABLE categoria AUTO_INCREMENT = 1;',
            'ALTER TABLE tipo_taxi AUTO_INCREMENT = 1;',
            'ALTER TABLE taxi_stg AUTO_INCREMENT = 1;'
        ]

        for query in reset_auto_increment_queries:
            cursor.execute(query)

        # Confirmar los cambios
        conexion.commit()

        print("Base de datos limpiada exitosamente.")

    except Exception as e:
        print(f"Error al limpiar la base de datos: {e}")

    finally:
        # Reactivar las restricciones de clave externa
        print(".....")

In [10]:
eliminar_datos_base(conexion)

Base de datos limpiada exitosamente.
.....


In [11]:
# INSERTAR DATOS EN LA TABLA TAXI_STG

def insert_taxi_stg(conexion, df):
    cursor = conexion.cursor()

    insert_query = "INSERT INTO taxi_stg(placa,clase,marca, ano_fabricacion,categoria,tipo_taxi) VALUES (%s, %s, %s, %s, %s, %s)"

    try:
        for index, row in df.iterrows():
            taxi_data = (row['Placa'], row['Clase'], row['Marca'], row['Año de fabricación del taxi'], row['Categoría'], row['Tipo taxi'])
            cursor.execute(insert_query, taxi_data)

        conexion.commit()
        print(f"Datos insertados correctamente")

    except Exception as e:
        print(f"Error al insertar datos: {e}")

    finally:
        cursor.close()

In [12]:
insert_taxi_stg(conexion, df)

Datos insertados correctamente


In [14]:
# LIMPIAR DATOS INNECESARIOS DE TAXI

def limpiar_datos_taxi_stg(conexion):
    try:
        # Desactivar las restricciones de clave externa para permitir la eliminación
        with conexion.cursor() as cursor:
            cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

        # Eliminar datos de las tablas
        delete_queries = [
            'delete from taxi_stg where ano_fabricacion=0;',
            'delete from taxi_stg where marca like"%Marca%no%registrada%";',
            'delete from taxi_stg where categoria like"%Categoria%no%registrada%";',
            'delete from taxi_stg where tipo_taxi="Noid"'        
        ]

        for query in delete_queries:
            with conexion.cursor() as cursor:
                cursor.execute(query)

        # Confirmar los cambios
        conexion.commit()

        print("Datos tabla taxi stg limpiada exitosamente.")

    except Exception as e:
        print(f"Error al limpiar la base de datos: {e}")

    finally:
        # Reactivar las restricciones de clave externa
        with conexion.cursor() as cursor:
            cursor.execute("SET FOREIGN_KEY_CHECKS=1;")

In [15]:
limpiar_datos_taxi_stg(conexion)

Datos tabla taxi stg limpiada exitosamente.


In [13]:
# OBTENER VALORES DISTINTOS Y ASIGNAR A LAS RESPECTIVAS TABLAS 

def obtener_distinct_values(conexion, column_name):
    try:
        with conexion.cursor() as cursor:
            select_query = f"SELECT DISTINCT({column_name}) FROM taxi_stg"
            cursor.execute(select_query)
            distinct_values = cursor.fetchall()
            print(f"Datos extraidos correctamente de: {column_name}")
            return tuple(value[0] for value in distinct_values)

    except Exception as e:
        print(f"Error al obtener valores distintos: {e}")
        return []

In [14]:
distinct_values_categorias=obtener_distinct_values(conexion, "categoria")

Datos extraidos correctamente de: categoria


In [15]:
distinct_values_tipo_taxi=obtener_distinct_values(conexion, "tipo_taxi")

Datos extraidos correctamente de: tipo_taxi


In [16]:
distinct_values_clases=obtener_distinct_values(conexion, "clase")

Datos extraidos correctamente de: clase


In [17]:
distinct_values_marcas=obtener_distinct_values(conexion, "marca")

Datos extraidos correctamente de: marca


In [18]:
# INSERTAR DATOS EXTRAIDOS EN LAS RESPECTIVAS TABLAS DE LA BASE DE DATOS

def insert_distinct_values_into_table(conexion, distinct_values, table_name, column_name):
        try:
            with conexion.cursor() as cursor:
                insert_query = f"INSERT INTO {table_name} ({column_name}) VALUES (%s)"
                for value in distinct_values:
                    cursor.execute(insert_query, (value,))

                conexion.commit()
                print(f"Datos insertados correctamente en {table_name}.")

        except Exception as e:
            print(f"Error al insertar datos en {table_name}: {e}")  

In [19]:
insert_distinct_values_into_table(conexion, distinct_values_marcas, "marca", "descripcion_marca")

Datos insertados correctamente en marca.


In [20]:
insert_distinct_values_into_table(conexion, distinct_values_clases, "clase", "descripcion_clase")

Datos insertados correctamente en clase.


In [21]:
insert_distinct_values_into_table(conexion, distinct_values_categorias, "categoria", "descripcion_categoria")

Datos insertados correctamente en categoria.


In [22]:
insert_distinct_values_into_table(conexion, distinct_values_tipo_taxi, "tipo_taxi", "descripcion_tipo_taxi")

Datos insertados correctamente en tipo_taxi.
